In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch_directml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
data = pd.read_csv("heart.csv")

# Features and target
independent = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
               'exang', 'oldpeak', 'slope', 'ca', 'thal']
dependent = ['target']

X = data[independent].values
y = data[dependent].values

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
device = torch_directml.device()
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Neural Network Model
class HeartDiseaseModel(nn.Module):
    def __init__(self):
        super(HeartDiseaseModel, self).__init__()
        self.fc1 = nn.Linear(len(independent), 16)  # Input Layer
        self.fc2 = nn.Linear(16, 8)  # Hidden Layer
        self.fc3 = nn.Linear(8, 1)   # Output Layer
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))  # Output between 0 and 1
        return x

# Model, Loss, Optimizer
model = HeartDiseaseModel().to(device)
criterion = nn.BCELoss()  # Binary Cross Entropy for classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training Loop
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Model Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).cpu().numpy()
    y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary
    accuracy = (y_pred == y_test).mean()
    print(f'Accuracy: {accuracy:.4f}')

# Save model after training
torch.save(model.state_dict(), "heart_disease_model.pth")
print("Model saved successfully.")




Dropped Escape call with ulEscapeCode : 0x03007703
/home/shiva/Projects/venv/lib/python3.12/site-packages/torch/nn/functional.py:3172: UserWarning: The operator 'aten::binary_cross_entropy' is not currently supported on the DML backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /__w/1/s/pytorch-directml-plugin/torch_directml/csrc/dml/dml_cpu_fallback.cpp:15.)
  return torch._C._nn.binary_cross_entropy(input, target, weight, reduction_enum)


Epoch [10/100], Loss: 0.5872
Epoch [20/100], Loss: 0.4017
Epoch [30/100], Loss: 0.2890
Epoch [40/100], Loss: 0.2609
Epoch [50/100], Loss: 0.2329
Epoch [60/100], Loss: 0.2057
Epoch [70/100], Loss: 0.1742
Epoch [80/100], Loss: 0.1404
Epoch [90/100], Loss: 0.1057
Epoch [100/100], Loss: 0.0776
Accuracy: 0.9366
Model saved successfully.
